# Yelp API Restaurant Calls By GPS Coordinates
Hypotenuse theory: testing a way to get the most restaurants in the bay area with minimal overlapping
For each coordinate listed in grid_coordinates.csv, return first 1k restaurants found via yelp api and
extract data regarding:

- name
- address & zip
- coordinates
- rating
- review count
- price level
- category
- yelp id

Write data to restaurant_data_coord.csv

In [50]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from api_keys1 import yelp_loba_api

### Perform API calls

- Dataframe from grid_coordinates.csv
- Build function that calls and writes restaurant data to new csv, given pair of coordinates
- Run list of coordinates through function

In [51]:
# Create dataframe from csv file
coords_df = pd.read_csv("Resources/grid_coordinates.csv", 
                        names=["Lat", "Lng"])
coords_df.head()

,Lat,Lng
0,38.871657,-123.109843
1,38.871657,-123.017222
2,38.871657,-122.924601
3,38.871657,-122.831980
4,38.871657,-122.739359


In [52]:
# Function returns up to 1000 restaurant listings for input coordinates
def get_restaurants(lat, lng, yelp_loba_api):
    
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": "Bearer %s" % yelp_loba_api}
    restaurant_data = []
    count = 0
    
    # Increases search return limit
    for offset in range(0, 1000, 50):
        
        # Set parameters and pass into API calls, radius 4023 meters = 2.5 miles
        params = {"term":"restaurants", "latitude":lat, "longitude":lng, "radius": 5690, 
                  "limit":50, "offset":offset}
        req = requests.get(url, params=params, headers=headers)
        
        # Breaks if error occurs with search
        if req.status_code == 400:
            break
        elif req.status_code == 200:
            
            # Convert to json
            response = req.json()
            
            # Log history
            count += 1
            print(f"Now processing query set {count} of max 20")
            
            # Breaks if no further entries in query
            if response["businesses"] == []:
                break
            
            else:
                # Iterate through business results and extract data
                for biz in response["businesses"]:
                    
                    # Logic to replace missing price level data with NaN
                    if "price" not in biz:
                        
                        # Replace both price & missing category data with empty string
                        if biz["categories"] == []:
                            restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], 
                                                    biz["coordinates"]["latitude"], 
                                                    biz["coordinates"]["longitude"], 
                                                    biz["location"]["address1"], 
                                                    biz["location"]["zip_code"], biz["rating"], 
                                                    biz["review_count"], "", "", biz["id"]])
                        else:
                            restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], 
                                                    biz["coordinates"]["latitude"], 
                                                    biz["coordinates"]["longitude"], 
                                                    biz["location"]["address1"], 
                                                    biz["location"]["zip_code"], biz["rating"], 
                                                    biz["review_count"], "", biz["categories"][0]["title"], 
                                                    biz["id"]])

                    # Replace missing category data with empty string
                    elif biz["categories"] == []:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], "", biz["id"]])

                    else:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], 
                                                biz["categories"][0]["title"], biz["id"]])

    # Write to csv
    with open('Resources/restaurant_data_coords_reduced5690radius.csv', 'a', encoding="utf-8") as csvFile:
        writer = csv.writer(csvFile)
        for restaurant in restaurant_data:
            writer.writerow(restaurant)
    csvFile.close()
    
    
    # Returns total count of restaurants in coordinates
    return response["total"]


In [53]:
# Track # of coordinates processed
count = 0

# List to track total restaurants found in coordinate
totals_count = []

print("LOG HISTORY OF API CALLS:")
print("---------------------------")

# Loop thru list of coordinates
lat_coords = [lat for lat in coords_df["Lat"]]
lng_coords = [lng for lng in coords_df["Lng"]]

for index in range(len(lat_coords)):
                   
    #  Call get_restaurants fn and append to total_count list
    totals_count.append(get_restaurants(lat_coords[index], lng_coords[index] , yelp_loba_api))
    
    # Print log history
    count += 1
    rem = len(lat_coords) - count
    print("-----------------------------------------")
    if count == len(lat_coords):
        print("Full list of coordinates processed!")
    elif count == len(lat_coords) - 1:
        print("Now getting results for final coordinates. Almost there!")
        print("-----------------------------------------")
    else:
        print(f"Data retrieval for coordinates complete")
        print(f"Getting results for next coordinates.. there are {rem} coordinates left")
        print("-----------------------------------------")

coords_df["Total # of Restaurants"] = totals_count

LOG HISTORY OF API CALLS:
---------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 531 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 530 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 529 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 528 coordinates left
---------------------------

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 498 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 497 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 496 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 495 coordinates left
-----------------------------------------
No

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 466 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 465 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 464 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 463 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
No

Now processing query set 9 of max 20
Now processing query set 10 of max 20
Now processing query set 11 of max 20
Now processing query set 12 of max 20
Now processing query set 13 of max 20
Now processing query set 14 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 438 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
Now processing query set 6 of max 20
Now processing query set 7 of max 20
Now processing query set 8 of max 20
Now processing query set 9 of max 20
Now processing query set 10 of max 20
Now processing query set 11 of max 20
Now processing query set 12 of max 20
Now processing query set 13 of max 20
Now processing query set 14 of max 20
Now processing query set 15 of max 20
Now processing que

Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 414 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
Now processing query set 6 of max 20
Now processing query set 7 of max 20
Now processing query set 8 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 413 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting resu

...

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 135 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 134 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 133 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 132 coordinates left
-----------------------------------------
No

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 110 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 109 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 108 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 107 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
---------------------------------------

Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 83 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 82 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
Now processing query set 6 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 81

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 51 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 50 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 49 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 48 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
D

Now processing query set 3 of max 20
Now processing query set 4 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 20 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 19 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 18 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 17 coordinates left
-----------------------------------------
Now pr

In [54]:
# Show coords_df with updated total # of restaurants
coords_df.head()


,Lat,Lng,Total # of Restaurants
0,38.871657,-123.109843,0
1,38.871657,-123.017222,5
2,38.871657,-122.924601,0
3,38.871657,-122.831980,0
4,38.871657,-122.739359,5


In [56]:
coords_df.sort_values(['Total # of Restaurants'], ascending=[False]).head(20)

,Lat,Lng,Total # of Restaurants
292,37.784307,-122.461496,2700
293,37.784307,-122.368875,2300
311,37.711817,-122.461496,1800
275,37.856797,-122.276254,1600
412,37.349367,-121.905770,1300
294,37.784307,-122.276254,1200
431,37.276877,-121.905770,1100
411,37.349367,-121.998391,1100
94,38.581697,-121.442665,927
312,37.711817,-122.368875,875


In [57]:
# Save restaurants counts to csv
coords_df.to_csv("Resources/total_count_coords_5690radius.csv")

### Create Dataframe

- Read csv file
- Clean data

In [58]:
# Read csv file
restaurants_df = pd.read_csv("Resources/restaurant_data_coords_reduced5690radius.csv", encoding = "ISO-8859-1", 
                            names=["search_ID", "Latitude_search", "Longitude_search", "Name", "Lat_Restaurant", "Lng_Restaurant", "Address", "Zip", 
                                   "Rating", "# of Reviews", "Price Level", "Category", "Yelp ID"])

# Replace NaN entries with blank string
restaurants_df = restaurants_df.fillna('')
restaurants_df

,search_ID,Latitude_search,Longitude_search,Name,Lat_Restaurant,Lng_Restaurant,Address,Zip,Rating,# of Reviews,Price Level,Category,Yelp ID
0,"38.871657, -123.017222",38.871657,-123.017222,Hamburger Ranch & Bar-B-Que,38.8178,-123.024,31195 N Redwood Hwy,95425,4.0,381,$$,Barbeque,0Dj4fW3J3DJzrI51PLadRA
1,"38.871657, -123.017222",38.871657,-123.017222,Cloverdale Ale Company,38.8056,-123.016,131 E 1st St,95425,4.0,21,$$,Pubs,wAbWk-do6kAUt3V9z8f9Wg
2,"38.871657, -123.017222",38.871657,-123.017222,La Hacienda,38.8065,-123.018,134 N Cloverdale Blvd,95425,3.5,83,$$,Mexican,aPow7ctyoPiaMGzHhzfZvQ
3,"38.871657, -123.017222",38.871657,-123.017222,Piacere Italiano Steak and Seafood,38.8107,-123.021,504 N Cloverdale Blvd,95425,3.5,122,$$,Italian,wcRhJxjChvs7Q4nBft_c8A
4,"38.871657, -123.017222",38.871657,-123.017222,MoE's Eagles Nest Deli,38.8057,-123.017,113 N Cloverdale Blvd,95425,3.5,61,$,Delis,mweczDf0CB-zt4JsremEPQ
5,"38.871657, -122.73935900000001",38.871657,-122.739359,Loch Lomond RoadHouse,38.8636,-122.718,10336 Loch Lomond Rd,95461,4.0,48,$$,American (Traditional),lKdiMPiuiKV-ILWD-XgXog
6,"38.871657, -122.73935900000001",38.871657,-122.739359,Infuse Coffee Bar,38.8634,-122.718,10336 Loch Lomond Rd,95461,5.0,4,,Coffee & Tea,_P0y4P7VZtE2pymBmMjJrw
7,"38.871657, -122.73935900000001",38.871657,-122.739359,Brick Oven Pizza,38.825,-122.722,16295 Hwy 175,95426,3.5,36,$$,Pizza,zZEOUAq-9jWHEXG-DSLZsg
8,"38.871657, -122.73935900000001",38.871657,-122.739359,Phat Salads and Wraps,38.3186,-122.3,,94558,4.0,48,$,Food Trucks,i6ZwMjYIbmFZ8EGYz6x5Bw
9,"38.871657, -122.73935900000001",38.871657,-122.739359,808 Island Grill,38.7369,-121.267,,95661,5.0,16,,Hawaiian,fLJLB8y-Bj9AEPVysp0JFQ


In [159]:
### Clean data, how to treat missing address, zip, price level, food trucks/mobile, low review counts
### Groupby on df